# **1. Data Cleaning**

# Setup

In [1]:
import pandas as pd

**Import data file: "Daily Stats Entry Form.xlsx"**

> This data file contains SHS Statistics data for Winter 2021



In [2]:
df_Raw0 = pd.read_excel("1_rawData/Daily Stats Entry Form.xlsx", index_col=False);
# df_Raw0[0:1]
df_Raw0[0:1][df_Raw0.columns.tolist()[:-1]]

,ID,Start time,Completion time,Email,Name,Please input the date of interaction with student,Please select the appropriate time period of your first interaction,Please select the appropriate Task Category or type it in the Other section
0,1,2021-01-14 08:30:05,2021-01-14 08:30:22,shs@douglascollege.ca,Students Helping Students,2021-01-04,07:30 - 08:00,Blackboard Issues


**Import data file: "Daily Stats Entry Form 1.xlsx"**

> This data file contains SHS Statistics data for Winter 2021 and early Summer 2021



In [3]:
df_Raw1 = pd.read_excel("1_rawData/Daily Stats Entry Form 1.xlsx", index_col=False);
df_Raw1 = df_Raw1.rename(columns={
    "Please input the Date of interaction with the Student":"Please input the date of interaction with student",
    "Please enter the source as the email address of Student (ex. xxx@student.douglascollege.ca)":"Please enter the source as the email address of Student (ex. murat@student.douglascollege.ca)"
    });
# df_Raw1[0:1]
df_Raw1[0:1][[
    "ID",
    "Start time",
    "Completion time",
    "Email",
    "Name",
    "Please input the date of interaction with student",
    "Please select the appropriate time period of your first interaction",
    "Please select the appropriate Task Category or type it in the Other section",
    "Is it Escalated to SD?"
]]

,ID,Start time,Completion time,Email,Name,Please input the date of interaction with student,Please select the appropriate time period of your first interaction,Please select the appropriate Task Category or type it in the Other section,Is it Escalated to SD?
0,1,2021-01-14 08:30:05,2021-01-14 08:30:22,shs@douglascollege.ca,Students Helping Students,2021-01-04,07:30 - 08:00,Blackboard Issues,NaN


**Import data file: "Daily Stats Entry Form Updated 2021.xlsx"**

> This data file contains SHS Statistics data for Summer 2021 and Fall 2021



In [4]:
df_Raw2 = pd.read_excel("1_rawData/Daily Stats Entry Form Updated 2021.xlsx", index_col=False);
# df_Raw2[0:1]
df_Raw2[0:1][df_Raw2.columns.tolist()[:-1]]

,Please input the Date of interaction with the Student,Please select the appropriate time period of your first interaction,Please select the appropriate Task Category or type it in the Other section,Is it Escalated to SD?
0,2021-07-02,20:00 - 20:30,Password Portal,No


# Data Cleaning

**Data Cleaning of Pre-Summer 2021 data** 
> SHS statistics data from Winter 2021 to early Summer 2021

Update column headings of dataframe

In [5]:
def updateColumns(dataframe_Raw):              
  # Reference: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html
  dataframe = dataframe_Raw.rename(columns={"Please input the date of interaction with student":"Date", 
                    "Please select the appropriate time period of your first interaction":"Time Period", 
                    "Please select the appropriate Task Category or type it in the Other section":"Task Category",
                    "Please enter the source as the email address of Student (ex. murat@student.douglascollege.ca)":"Student Contact"
                  });
  dataframe["Date"] = pd.to_datetime(dataframe["Date"]);
  dataframe["Start time"] = pd.to_datetime(dataframe["Start time"]);
  dataframe["Completion time"] = pd.to_datetime(dataframe["Completion time"]);
  return dataframe;

In [6]:
df_0 = updateColumns(df_Raw0);
print(f"Number of Rows: {len(df_0)}");
# df_0[0:1]
df_0[0:1][df_0.columns.tolist()[:-1]]

Number of Rows: 385


,ID,Start time,Completion time,Email,Name,Date,Time Period,Task Category
0,1,2021-01-14 08:30:05,2021-01-14 08:30:22,shs@douglascollege.ca,Students Helping Students,2021-01-04,07:30 - 08:00,Blackboard Issues


In [7]:
df_1 = updateColumns(df_Raw1);
df_1 = df_1[["ID","Start time","Completion time","Email","Name","Date","Time Period","Task Category","Student Contact","Is it Escalated to SD?"]];
print(f"Number of Rows: {len(df_1)}");
# df_1[0:1]
df_1[0:1][[
    "ID",
    "Start time",
    "Completion time",
    "Email",
    "Name",
    "Date",
    "Time Period",
    "Task Category",
    "Is it Escalated to SD?"
]]

Number of Rows: 587


,ID,Start time,Completion time,Email,Name,Date,Time Period,Task Category,Is it Escalated to SD?
0,1,2021-01-14 08:30:05,2021-01-14 08:30:22,shs@douglascollege.ca,Students Helping Students,2021-01-04,07:30 - 08:00,Blackboard Issues,NaN


Perform high level data cleaning and manipulation

In [8]:
# Reference: 
# - https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html
# - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop_duplicates.html
# - https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.reset_index.html#pandas.DataFrame.reset_index

df_temp = pd.concat([df_0, df_1], ignore_index=True);
# df_temp = df_temp.sort_values(["Start time","Completion time"], ascending=True);
df_temp = df_temp.sort_values(["Date","Time Period"], ascending=True);
df_temp = df_temp.drop_duplicates(keep="last");
df_temp = df_temp.reset_index();
for i in df_temp.index:
  if (df_temp.loc[i, "Is it Escalated to SD?"] != "Yes") & (df_temp.loc[i, "Is it Escalated to SD?"] != "No"):
    df_temp.loc[i, "Is it Escalated to SD?"] = "No";
df_temp = df_temp[df_temp.columns.tolist()[6:]];
df_temp = df_temp.rename(columns={"Time Period":"Time Range", "Is it Escalated to SD?":"Escalated to SD?"});
df_temp["Description"] = df_temp["Task Category"];
df_temp = df_temp[["Date","Time Range","Task Category","Escalated to SD?","Student Contact","Description"]];

Clean the Task Category column

In [9]:
for i in df_temp.index:
  # ************************* Zoom Help *************************
  if df_temp.loc[i, "Task Category"] == "Zoom Troubleshooting":
    df_temp.loc[i, "Task Category"] = "Zoom Help";
  
  if df_temp.loc[i, "Task Category"] == "Zoom Issue":
    df_temp.loc[i, "Task Category"] = "Zoom Help";
  
  if df_temp.loc[i, "Task Category"] == "Chrome PDF Plugin issue through Zoom":
    df_temp.loc[i, "Task Category"] = "Zoom Help";
  
  if df_temp.loc[i, "Task Category"] == "Troubleshoot Zoom":
    df_temp.loc[i, "Task Category"] = "Zoom Help";
  
  if df_temp.loc[i, "Task Category"] == "Troubleshoot zoom":
    df_temp.loc[i, "Task Category"] = "Zoom Help";
  
  # ************************* Others *************************
  if df_temp.loc[i, "Task Category"] == "Laptop Recommendation":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "File Extension Question":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "microphone problem":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "black screen":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Android Studio":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Consent Form Issue":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Uploading video to laptop":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "File association/unable to open":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "microphone":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Having trouble adding a pdf to a report":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "DS App":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Douglas College Mobile apps":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Looking for Windows license key":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Rotating a photo in a pdf":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Google Docs":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Chrome PDF plugin issue":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Laptop decision ":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Douglas Website":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Windows Language Settings":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "student whatsapp group advice":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Basic Technical Assistance":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "basic technical assistance":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Hardware recommendation for online classes":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "hardware/software advice seeking":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "carson.schumann8@gmail.com":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Issue with Adobe Acrobat Reader":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Can't reach college website":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "college website access":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "PDF issue":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "can't load college website":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Douglas website issue":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Basic Technical Assistance":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "website not loading/issue solved itself":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "Basic Technical assistance":
    df_temp.loc[i, "Task Category"] = "Others";
  
  if df_temp.loc[i, "Task Category"] == "student whatsapp group advise ":
    df_temp.loc[i, "Task Category"] = "Others";

  if df_temp.loc[i, "Task Category"] == "Website":
    df_temp.loc[i, "Task Category"] = "Others";
  
  # ************************* Bookstore *************************
  if df_temp.loc[i, "Task Category"] == "Textbook Recommendation":
    df_temp.loc[i, "Task Category"] = "Bookstore";
  
  if df_temp.loc[i, "Task Category"] == "Coursebook material access":
    df_temp.loc[i, "Task Category"] = "Bookstore";
  
  if df_temp.loc[i, "Task Category"] == "Coursepack issue":
    df_temp.loc[i, "Task Category"] = "Bookstore";
  
  if df_temp.loc[i, "Task Category"] == "Adobe Coursepack":
    df_temp.loc[i, "Task Category"] = "Bookstore";
  
  # ************************* Library and Learning Centre *************************
  if df_temp.loc[i, "Task Category"] == "Computer Lab Availability":
    df_temp.loc[i, "Task Category"] = "Library and Learning Centre";
  
  # ************************* Lockdown Browser *************************
  if df_temp.loc[i, "Task Category"] == "Lockdown download":
    df_temp.loc[i, "Task Category"] = "Lockdown Browser";
  
  if df_temp.loc[i, "Task Category"] == "Zoom and Lockdown Browser issue":
    df_temp.loc[i, "Task Category"] = "Lockdown Browser";
  
  if df_temp.loc[i, "Task Category"] == "Zoom and Lockdown browser Session issue":
    df_temp.loc[i, "Task Category"] = "Lockdown Browser";
  
  # ************************* Microsoft 365 Issues *************************
  if df_temp.loc[i, "Task Category"] == "OneNote Collab":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Word to PDF conversion":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Microsoft Office issues":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Microsoft Excel Issue":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Azure Issue":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";

  if df_temp.loc[i, "Task Category"] == "Microsoft Azure account restricted":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Microsoft Azure":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Microsoft Word issue":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Need help in making a powerpoint YouTube video":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Word Issue":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Issue with Microsoft Powerpoint":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_temp.loc[i, "Task Category"] == "Issue with Powerpoint":
    df_temp.loc[i, "Task Category"] = "Microsoft 365 Issues";

  # ************************* Blackboard Issues *************************
  if df_temp.loc[i, "Task Category"] == "Assignment Submission Issue":
    df_temp.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_temp.loc[i, "Task Category"] == "Having trouble completing the AIE module":
    df_temp.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_temp.loc[i, "Task Category"] == "Learn Blackboard":
    df_temp.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_temp.loc[i, "Task Category"] == "Blackboard - General (Course content etc)":
    df_temp.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_temp.loc[i, "Task Category"] == "Blackboard - Technical Assistance (audio/video)":
    df_temp.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_temp.loc[i, "Task Category"] == "can't load videos on blackboard":
    df_temp.loc[i, "Task Category"] = "Blackboard Issues";
  
  # ************************* MyAccount Issues *************************
  if df_temp.loc[i, "Task Category"] == "tax forms":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "Unofficial Transcript":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "registering personal email in myaccount":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "myaccount issue":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "access to courses":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "Mypath issue,Basic technical assistance":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "logs out instantly after login to MyAccount":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "MyAccount - Missing CNA Email":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "MyAccount - Directed student to SSPR":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_temp.loc[i, "Task Category"] == "mypath issue":
    df_temp.loc[i, "Task Category"] = "MyAccount Issues";
  
  # ************************* Academic Advising *************************
  if df_temp.loc[i, "Task Category"] == "Graduation Details":
    df_temp.loc[i, "Task Category"] = "Academic Advising";
  
  if df_temp.loc[i, "Task Category"] == "Graduation Application Issue":
    df_temp.loc[i, "Task Category"] = "Academic Advising";
  
  if df_temp.loc[i, "Task Category"] == "Advising Services":
    df_temp.loc[i, "Task Category"] = "Academic Advising";
  
  if df_temp.loc[i, "Task Category"] == "First Class Instruction":
    df_temp.loc[i, "Task Category"] = "Academic Advising";
  
  if df_temp.loc[i, "Task Category"] == "First Class Instructions":
    df_temp.loc[i, "Task Category"] = "Academic Advising";
  
  if df_temp.loc[i, "Task Category"] == "First Class issue":
    df_temp.loc[i, "Task Category"] = "Academic Advising";
  
  if df_temp.loc[i, "Task Category"] == "First class issue":
    df_temp.loc[i, "Task Category"] = "Academic Advising";

  # ************************* Printing *************************
  if df_temp.loc[i, "Task Category"] == "Digitalizing written documents":
    df_temp.loc[i, "Task Category"] = "Printing";
  
  # ************************* Enrollment Service *************************
  if df_temp.loc[i, "Task Category"] == "Summer semester dates":
    df_temp.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_temp.loc[i, "Task Category"] == "Orientation":
    df_temp.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_temp.loc[i, "Task Category"] == "Registration":
    df_temp.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_temp.loc[i, "Task Category"] == "Verification Of Enrollment Issue":
    df_temp.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_temp.loc[i, "Task Category"] == "Verification of Enrollment Issue":
    df_temp.loc[i, "Task Category"] = "Enrollment Service";
  
  # ************************* CNA/Login Issues *************************
  if df_temp.loc[i, "Task Category"] == "MyPath Login Issue":
    df_temp.loc[i, "Task Category"] = "CNA/Login Issues";
  
  if df_temp.loc[i, "Task Category"] == "CNA Questions":
    df_temp.loc[i, "Task Category"] = "CNA/Login Issues";
  
  if df_temp.loc[i, "Task Category"] == "Password Portal":
    df_temp.loc[i, "Task Category"] = "CNA/Login Issues";
  
  # ************************* Douglas College International *************************
  if df_temp.loc[i, "Task Category"] == "Working as an international student":
    df_temp.loc[i, "Task Category"] = "Douglas College International";
  
  if df_temp.loc[i, "Task Category"] == "Regarding Medical Insurance":
    df_temp.loc[i, "Task Category"] = "Douglas College International";
  
  # ************************* Kaltura *************************
  if df_temp.loc[i, "Task Category"] == "Kaltura issue":
    df_temp.loc[i, "Task Category"] = "Kaltura";
  
  if df_temp.loc[i, "Task Category"] == "Kaltura Issue":
    df_temp.loc[i, "Task Category"] = "Kaltura";
  
  if df_temp.loc[i, "Task Category"] == "Kaltura Media Upload":
    df_temp.loc[i, "Task Category"] = "Kaltura";
  
  # ************************* Citrix *************************
  if df_temp.loc[i, "Task Category"] == "Sage 50":
    df_temp.loc[i, "Task Category"] = "Citrix";

df_temp["Task Category"].unique().tolist()

['Academic Advising',
 'Blackboard Issues',
 'MyAccount Issues',
 'Zoom Help',
 'Bookstore',
 'CNA/Login Issues',
 'Others',
 'Enrollment Service',
 'Microsoft 365 Issues',
 'Library and Learning Centre',
 'Citrix',
 'Lockdown Browser',
 'Kaltura',
 'Douglas College International',
 'Printing']

Ouptut of cleaned dataframe of Pre-Summer 2021 SHS Statistics data

In [10]:
# df_temp[0:1]
df_temp[0:1][[
    "Date",
    "Time Range",
    "Task Category",
    "Escalated to SD?",
    "Description"
]]

,Date,Time Range,Task Category,Escalated to SD?,Description
0,2021-01-04,07:00 - 07:30,Academic Advising,No,Academic Advising


**Data Cleaning of Post-Winter 2021 data**
> SHS statistics data from Summer 2021 to Fall 2021

Update column headings of the dataframe

In [11]:
df_2 = df_Raw2.copy();
df_2 = df_2.rename(columns={"Please input the Date of interaction with the Student":"Date", 
                            "Please select the appropriate time period of your first interaction":"Time Range", 
                            "Please select the appropriate Task Category or type it in the Other section":"Task Category",
                            "Is it Escalated to SD?":"Escalated to SD?",
                            "    Please enter the source as the email address of Student (ex. xxx@student.douglascollege.ca)":"Student Contact"});
df_2["Date"] = pd.to_datetime(df_2["Date"]);
df_2["Description"] = df_2["Task Category"];
df_2 = df_2.sort_values(["Date","Time Range"], ascending=True);
print(f"Number of Rows: {len(df_2)}");
# df_2[0:1]
df_2[0:1][[
    "Date",
    "Time Range",
    "Task Category",
    "Escalated to SD?",
    "Description"
]]

Number of Rows: 595


,Date,Time Range,Task Category,Escalated to SD?,Description
82,2021-06-28,07:00 - 07:30,Hardware Issue,No,Hardware Issue


Clean the Time Range column

In [12]:
for i in df_2.index:
  if df_2.loc[i, "Time Range"] == "8:30 - 9:00":
    df_2.loc[i, "Time Range"] = "08:30 - 09:00";
  
  if df_2.loc[i, "Time Range"] == "07:00 - 7:30":
    df_2.loc[i, "Time Range"] = "07:00 - 07:30";
  
  if df_2.loc[i, "Time Range"] == "7:00 - 7:30":
    df_2.loc[i, "Time Range"] = "07:00 - 07:30";
  
  if df_2.loc[i, "Time Range"] == "9:30 - 10:30":
    df_2.loc[i, "Time Range"] = "09:30 - 10:30";
  
  if df_2.loc[i, "Time Range"] == "8:00 - 8:30":
    df_2.loc[i, "Time Range"] = "08:00 - 08:30";
  
  if df_2.loc[i, "Time Range"] == "8:00 - 8:30":
    df_2.loc[i, "Time Range"] = "08:00 - 08:30";
  
  if df_2.loc[i, "Time Range"] == "7:30 - 8:00":
    df_2.loc[i, "Time Range"] = "07:30 - 08:00";
  
  if df_2.loc[i, "Time Range"] == "16:00-16:30":
    df_2.loc[i, "Time Range"] = "16:00 - 16:30";
  
  if df_2.loc[i, "Time Range"] == "18:00-18:30":
    df_2.loc[i, "Time Range"] = "18:00 - 18:30";
  
  if df_2.loc[i, "Time Range"] == "6:30 - 7:00":
    df_2.loc[i, "Time Range"] = "06:30 - 07:00";
  
  if df_2.loc[i, "Time Range"] == "9:30 - 10:00":
    df_2.loc[i, "Time Range"] = "09:30 - 10:00";
  
  if df_2.loc[i, "Time Range"] == "10:00-10:30":
    df_2.loc[i, "Time Range"] = "10:00 - 10:30";
  
  if df_2.loc[i, "Time Range"] == "7:30 - 8:30":
    df_2.loc[i, "Time Range"] = "07:30 - 08:30";
  
  if df_2.loc[i, "Time Range"] == "9:00 - 9:30":
    df_2.loc[i, "Time Range"] = "09:00 - 09:30";
  
  if df_2.loc[i, "Time Range"] == "7:00-7:30":
    df_2.loc[i, "Time Range"] = "07:00 - 07:30";
  
  if df_2.loc[i, "Time Range"] == "10:30-11:00":
    df_2.loc[i, "Time Range"] = "10:30 - 11:00";
  
  if df_2.loc[i, "Time Range"] == "8:30-9:00":
    df_2.loc[i, "Time Range"] = "08:30 - 09:00";
  
  if df_2.loc[i, "Time Range"] == "13:00 -13:30":
    df_2.loc[i, "Time Range"] = "13:00 - 13:30";
  
  if df_2.loc[i, "Time Range"] == "14:00 -14:30":
    df_2.loc[i, "Time Range"] = "14:00 - 14:30";
  
  if df_2.loc[i, "Time Range"] == "15:30 -16:00":
    df_2.loc[i, "Time Range"] = "15:30 - 16:00";
  
  if df_2.loc[i, "Time Range"] == "8:00-8:30":
    df_2.loc[i, "Time Range"] = "08:00 - 08:30";
  
  if df_2.loc[i, "Time Range"] == "15:00 -15:30":
    df_2.loc[i, "Time Range"] = "15:00 - 15:30";
  
  if df_2.loc[i, "Time Range"] == "12:30-13:00":
    df_2.loc[i, "Time Range"] = "12:30 - 13:00";
  
  if df_2.loc[i, "Time Range"] == "14:30-15:00":
    df_2.loc[i, "Time Range"] = "14:30 - 15:00";
  
  if df_2.loc[i, "Time Range"] == "9:00-9:30":
    df_2.loc[i, "Time Range"] = "09:00 - 09:30";
  
  if df_2.loc[i, "Time Range"] == "09:30-10:00":
    df_2.loc[i, "Time Range"] = "09:30 - 10:00";
  
  if df_2.loc[i, "Time Range"] == "9:30-10:00":
    df_2.loc[i, "Time Range"] = "09:30 - 10:00";
  
  if df_2.loc[i, "Time Range"] == "11:30-12:00":
    df_2.loc[i, "Time Range"] = "11:30 - 12:00";
  
  if df_2.loc[i, "Time Range"] == "16:30-17:00":
    df_2.loc[i, "Time Range"] = "16:30 - 17:00";
  
  if df_2.loc[i, "Time Range"] == "18:00 -18:30":
    df_2.loc[i, "Time Range"] = "18:00 - 18:30";
  
  if df_2.loc[i, "Time Range"] == "12:00-12:30":
    df_2.loc[i, "Time Range"] = "12:00 - 12:30";
  
  if df_2.loc[i, "Time Range"] == "13:30-14:00":
    df_2.loc[i, "Time Range"] = "13:30 - 14:00";
  
  if df_2.loc[i, "Time Range"] == "10:30 - 11:30":
    df_2.loc[i, "Time Range"] = "10:30 - 11:00";
  
  if df_2.loc[i, "Time Range"] == "10:30-11:30":
    df_2.loc[i, "Time Range"] = "10:30 - 11:00";
  
  if df_2.loc[i, "Time Range"] == "7:30-8:00":
    df_2.loc[i, "Time Range"] = "07:30 - 08:00";
  
  if df_2.loc[i, "Time Range"] == "15:00-15:30":
    df_2.loc[i, "Time Range"] = "15:00 - 15:30";
  
  if df_2.loc[i, "Time Range"] == "07:00 -07:30":
    df_2.loc[i, "Time Range"] = "07:00 - 07:30";
  
  if df_2.loc[i, "Time Range"] == "07:30 -08:00":
    df_2.loc[i, "Time Range"] = "07:30 - 08:00";
  
  if df_2.loc[i, "Time Range"] == "11:00 -11:30":
    df_2.loc[i, "Time Range"] = "11:00 - 11:30";
  
  if df_2.loc[i, "Time Range"] == "13:00 - 13:00":
    df_2.loc[i, "Time Range"] = "13:00 - 13:30";
  
  if df_2.loc[i, "Time Range"] == "15:30 - 15:30":
    df_2.loc[i, "Time Range"] = "15:30 - 16:00";
  
  if df_2.loc[i, "Time Range"] == "18:30 - 18:30":
    df_2.loc[i, "Time Range"] = "18:30 - 19:00";
  
  if df_2.loc[i, "Time Range"] == "10:00 - 11:00":
    df_2.loc[i, "Time Range"] = "10:00 - 10:30";
  
  if df_2.loc[i, "Time Range"] == "20:30 - 21:30":
    df_2.loc[i, "Time Range"] = "20:30 - 21:00";
  
  if df_2.loc[i, "Time Range"] == "07:30 - 08:30":
    df_2.loc[i, "Time Range"] = "07:30 - 08:00";
  
  if df_2.loc[i, "Time Range"] == "14:30 - 15:30":
    df_2.loc[i, "Time Range"] = "14:30 - 15:00";
  
  if df_2.loc[i, "Time Range"] == "09:30 - 10:30":
    df_2.loc[i, "Time Range"] = "09:30 - 10:00";
  
  if df_2.loc[i, "Time Range"] == "06:30 - 07:00":
    df_2.loc[i, "Time Range"] = "07:00 - 07:30";
  
df_2["Time Range"].unique().tolist()

['07:00 - 07:30',
 '09:00 - 09:30',
 '20:00 - 20:30',
 '08:00 - 08:30',
 '08:30 - 09:00',
 '11:30 - 12:00',
 '15:30 - 16:00',
 '16:30 - 17:00',
 '17:30 - 18:00',
 '12:30 - 13:00',
 '14:30 - 15:00',
 '10:30 - 11:00',
 '13:00 - 13:30',
 '18:00 - 18:30',
 '07:30 - 08:00',
 '09:30 - 10:00',
 '15:00 - 15:30',
 '10:00 - 10:30',
 '11:00 - 11:30',
 '19:30 - 20:00',
 '12:00 - 12:30',
 '18:30 - 19:00',
 '14:00 - 14:30',
 '20:30 - 21:00',
 '17:00 - 17:30',
 '13:30 - 14:00',
 '19:00 - 19:30',
 '16:00 - 16:30']

Clean the Task Category column

In [13]:
for i in df_2.index:
  # ******************* CNA/Login Issues ***********************
  if df_2.loc[i, "Task Category"] == "Password Portal":
    df_2.loc[i, "Task Category"] = "CNA/Login Issues";
  
  if df_2.loc[i, "Task Category"] == "Career Hub Issue":
    df_2.loc[i, "Task Category"] = "CNA/Login Issues";
  
  if df_2.loc[i, "Task Category"] == "CNA Questions":
    df_2.loc[i, "Task Category"] = "CNA/Login Issues";
  
  if df_2.loc[i, "Task Category"] == "Student lost CNA Credentials":
    df_2.loc[i, "Task Category"] = "CNA/Login Issues";
  
  # ******************* Blackboard Issues ***********************
  if df_2.loc[i, "Task Category"] == "Blackboard - General (Course content etc)":
    df_2.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_2.loc[i, "Task Category"] == "Blackboard App Issue":
    df_2.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_2.loc[i, "Task Category"] == "Blackboard App issue":
    df_2.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_2.loc[i, "Task Category"] == "Blackboard - Technical Assistance (audio/video)":
    df_2.loc[i, "Task Category"] = "Blackboard Issues";

  if df_2.loc[i, "Task Category"] == "Blackboard - error":
    df_2.loc[i, "Task Category"] = "Blackboard Issues";
  
  if df_2.loc[i, "Task Category"] == "Blackboard-Mail ":
    df_2.loc[i, "Task Category"] = "Blackboard Issues";

  # ******************* MyAccount Issues ***********************
  if df_2.loc[i, "Task Category"] == "MyAccount - Directed student to SSPR":
    df_2.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_2.loc[i, "Task Category"] == "Can't log into MyAccount":
    df_2.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_2.loc[i, "Task Category"] == "MyAccount Locked":
    df_2.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_2.loc[i, "Task Category"] == "MyAccount Disabled":
    df_2.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_2.loc[i, "Task Category"] == "MyAccount Questions":
    df_2.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_2.loc[i, "Task Category"] == "MyAccount - Missing CNA Email":
    df_2.loc[i, "Task Category"] = "MyAccount Issues";
  
  if df_2.loc[i, "Task Category"] == "Myaccount issue":
    df_2.loc[i, "Task Category"] = "MyAccount Issues";
  
  # ******************* Enrollment Service ***********************
  if df_2.loc[i, "Task Category"] == "Registration":
    df_2.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_2.loc[i, "Task Category"] == "Fee Payment":
    df_2.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_2.loc[i, "Task Category"] == "Enrollment":
    df_2.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_2.loc[i, "Task Category"] == "Upass and change of course":
    df_2.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_2.loc[i, "Task Category"] == "Enrolment Service":
    df_2.loc[i, "Task Category"] = "Enrollment Service";
  
  if df_2.loc[i, "Task Category"] == "Admissions":
    df_2.loc[i, "Task Category"] = "Enrollment Service";
  
  # ******************* Kaltura ***********************
  if df_2.loc[i, "Task Category"] == "Kaltura Upload":
    df_2.loc[i, "Task Category"] = "Kaltura";
  
  # ******************* Academic Advising ***********************
  if df_2.loc[i, "Task Category"] == "Advising Services":
    df_2.loc[i, "Task Category"] = "Academic Advising";
  
  # ******************* Microsoft 365 Issues ***********************
  if df_2.loc[i, "Task Category"] == "PowerPoint Issue":
    df_2.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  if df_2.loc[i, "Task Category"] == "PowerPoint Help":
    df_2.loc[i, "Task Category"] = "Microsoft 365 Issues";
  
  # ******************* Others ***********************
  if df_2.loc[i, "Task Category"] == "Hardware Issue":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "No":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "Not Specified":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "PC Recommendation":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "Course Software Technical Assistance":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "Microsoft Windows Issues":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "Computer Hardware Issues":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "Other":
    df_2.loc[i, "Task Category"] = "Others";
  
  if df_2.loc[i, "Task Category"] == "Computer Software Issues":
    df_2.loc[i, "Task Category"] = "Others";
  
  # ******************* Printing ***********************
  if df_2.loc[i, "Task Category"] == "Scanner Question":
    df_2.loc[i, "Task Category"] = "Printing";
  
  # ******************* WiFi Connectivity ***********************
  if df_2.loc[i, "Task Category"] == "College Wifi Issues":
    df_2.loc[i, "Task Category"] = "WiFi Connectivity";
  
  # ******************* Library and Learning Centre ***********************
  if df_2.loc[i, "Task Category"] == "Learning centre":
    df_2.loc[i, "Task Category"] = "WiFi Connectivity";
  
  if df_2.loc[i, "Task Category"] == "Help with assisgnments ":
    df_2.loc[i, "Task Category"] = "WiFi Connectivity";

df_2["Task Category"].unique().tolist()

['Others',
 'MyAccount Issues',
 'Microsoft 365 Issues',
 'CNA/Login Issues',
 'Academic Advising',
 'Blackboard Issues',
 'Enrollment Service',
 'Kaltura',
 'Zoom Help',
 'Printing',
 'Citrix',
 'Douglas College International',
 'WiFi Connectivity',
 'Bookstore']

Clean the 'Escalated to SD?' column

In [14]:
for i in df_2.index:
  if df_2.loc[i, "Escalated to SD?"] == "no":
    df_2.loc[i, "Escalated to SD?"] = "No";

df_2["Escalated to SD?"].unique().tolist()

['No', 'Yes']

Ouptut of cleaned dataframe of Post-Winter 2021 SHS Statistics data

In [15]:
print(f"Number of Rows: {len(df_2)}");
# df_2[0:1]
df_2[0:1][[
    "Date",
    "Time Range",
    "Task Category",
    "Escalated to SD?",
    "Description"
]]

Number of Rows: 595


,Date,Time Range,Task Category,Escalated to SD?,Description
82,2021-06-28,07:00 - 07:30,Others,No,Hardware Issue


**Combine Pre-Summer 2021 data and Post-Summer 2021 data together**

Perform high level dataframe manipulation

In [16]:
df = pd.concat([df_temp, df_2], ignore_index=True);
df = df.sort_values(["Date","Time Range"], ascending=True);
df = df.drop_duplicates(keep="last");
df = df.reset_index();
df = df[df.columns.tolist()[1:]];
df["Date"] = pd.to_datetime(df["Date"]);

Check to see if all dates in the dataframe is a legitimate workday/weekday

In [17]:
dayOfWeek = ["Monday","Tuesday","Wednesday","Thursday","Friday"];
for i in df.index:
  status = False;
  for day in dayOfWeek:
    if pd.Timestamp(df.loc[i, "Date"]).day_name() == day:
      status = True;
  if status == False:
    print(df.loc[i, "Date"]);

2021-02-27 00:00:00
2021-08-21 00:00:00


Manually adjust the dates so that they reflect a legitimate workday/weekday date

In [18]:
for i in df.index:
  if df.loc[i, "Date"] == pd.Timestamp("2021-02-27"):
    df.loc[i, "Date"] = pd.Timestamp("2021-02-26");
  
  if df.loc[i, "Date"] == pd.Timestamp("2021-08-21"):
    df.loc[i, "Date"] = pd.Timestamp("2021-08-20");

**Final Cleaned 2021 SHS Statistics Data**

In [19]:
print(f"Number of Rows: {len(df)}\tNumber of Columns: {len(df.columns.tolist())}");
# df.head()
df.head()[[
    "Date",
    "Time Range",
    "Task Category",
    "Escalated to SD?",
    "Description"
]]

Number of Rows: 1162	Number of Columns: 6


,Date,Time Range,Task Category,Escalated to SD?,Description
0,2021-01-04,07:00 - 07:30,Academic Advising,No,Academic Advising
1,2021-01-04,07:30 - 08:00,Blackboard Issues,No,Blackboard Issues
2,2021-01-04,07:30 - 08:00,Blackboard Issues,No,Blackboard Issues
3,2021-01-04,07:30 - 08:00,MyAccount Issues,No,MyAccount Issues
4,2021-01-04,08:00 - 08:30,Zoom Help,No,Zoom Troubleshooting


# Loading

**Load cleaned dataframe for analysis**

In [20]:
df.to_excel("2_data/Cleaned 2021 SHS Statistics Data.xlsx", index=False);